# Querying Knowledge Graphs with Cypher

### Import packages and set up Neo4

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd

from langchain_community.graphs import Neo4jGraph

_ = load_dotenv()

### Setting Environment Variables

In [2]:
os.environ["NEO4J_USERNAME"] = os.getenv("NEO4J_USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("NEO4J_PASSWORD")
os.environ["NEO4J_URI"] = os.getenv("NEO4J_URI")

In [3]:
kg = Neo4jGraph()

In [4]:
kg.refresh_schema()
print(kg.schema)

Node properties:
Game {rank: INTEGER, name: STRING, platform: STRING, year: INTEGER, genre: STRING, publisher: STRING, na_sales: FLOAT, eu_sales: FLOAT, jp_sales: FLOAT, other_sales: FLOAT, global_sales: FLOAT}
Platform {name: STRING}
Publisher {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Game)-[:RELEASED_ON]->(:Platform)
(:Game)-[:PUBLISHED_BY]->(:Publisher)
(:Game)-[:BELONGS_TO_GENRE]->(:Genre)


### Examples to show why Knowledge Graphs over SQL

**Example of Schema Flexibility**

Before

In [6]:
cypher = """
MATCH (game:Game {name:"Hades"})
RETURN game
"""

kg.query(cypher)

[]

In [7]:
cypher = """
CREATE (game:Game {name:"Hades"})
RETURN game
"""

kg.query(cypher)

[{'game': {'name': 'Hades'}}]

After

In [8]:
cypher = """
MATCH (game:Game {name:"Hades"})
RETURN game
"""

kg.query(cypher)

[{'game': {'name': 'Hades'}}]

Before giving it new properties, we will formulate relationship.

In [9]:
cypher = """
MATCH (hades:Game {name:"Hades"}), (rolePlaying:Genre {name:"Role-Playing"})
MERGE (hades)-[hasRelationship:BELONGS_TO_GENRE]->(rolePlaying)
RETURN hades, hasRelationship, rolePlaying
"""
kg.query(cypher)

[{'hades': {'name': 'Hades'},
  'hasRelationship': ({'name': 'Hades'},
   'BELONGS_TO_GENRE',
   {'name': 'Role-Playing'}),
  'rolePlaying': {'name': 'Role-Playing'}}]

In [10]:
cypher = """
MATCH (n:Game {name:"Hades"})
SET n.release_year = 2020,
    n.genre = "Role-Playing",
    n.developer = "Supergiant Games"
"""
kg.query(cypher)

[]

**Example of Query Simplicity and Readability**

Graph Example

In [11]:
cypher = """
MATCH (g:Game)-[:BELONGS_TO_GENRE]->(genre:Genre), 
      (g)-[:PUBLISHED_BY]->(publisher:Publisher)
WHERE publisher.name = 'Nintendo'
RETURN genre.name, SUM(g.global_sales) AS TotalSales
ORDER BY TotalSales DESC
"""
kg.query(cypher)

[{'genre.name': 'Platform', 'TotalSales': 427.21},
 {'genre.name': 'Role-Playing', 'TotalSales': 284.9},
 {'genre.name': 'Sports', 'TotalSales': 218.00999999999993},
 {'genre.name': 'Misc', 'TotalSales': 180.66999999999996},
 {'genre.name': 'Racing', 'TotalSales': 151.29999999999995},
 {'genre.name': 'Action', 'TotalSales': 128.17999999999995},
 {'genre.name': 'Puzzle', 'TotalSales': 124.88000000000002},
 {'genre.name': 'Shooter', 'TotalSales': 69.72999999999999},
 {'genre.name': 'Simulation', 'TotalSales': 60.510000000000005},
 {'genre.name': 'Fighting', 'TotalSales': 53.35000000000001},
 {'genre.name': 'Adventure', 'TotalSales': 35.709999999999994},
 {'genre.name': 'Strategy', 'TotalSales': 27.349999999999998}]

SQL Example

In [ ]:
"""SELECT genre.name, SUM(g.global_sales) AS TotalSales
FROM games g
JOIN genres genre ON g.genre_id = genre.id
JOIN publishers publisher ON g.publisher_id = publisher.id
WHERE publisher.name = 'Nintendo'
GROUP BY genre.name
ORDER BY TotalSales DESC;"""

Display relationship between all nodes

In [ ]:
# MATCH p=()-[]->() RETURN p; Run in cypher shell